In [5]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score


### About dataset

This is Kaggle dataset taken from Home Credit - Credit Risk Model Stability competition(https://www.kaggle.com/competitions/home-credit-credit-risk-model-stability).

Overview
The goal of this competition is to predict which clients are more likely to default on their loans. The evaluation will favor solutions that are stable over time.

Your participation may offer consumer finance providers a more reliable and longer-lasting way to assess a potential client’s default risk.

In [14]:
# Открываем файл для чтения с использованием контекстного менеджера
with open("data_path.txt", "r") as file:
    # Читаем содержимое файла
    dataPath = file.read()
dataPath

'D:/data/kaggle_comp1/'

In [15]:
file_name = "feature_definitions.csv"

file_path = os.path.join(dataPath, file_name)

# Прочитайте CSV файл в DataFrame с помощью pandas
df = pd.read_csv(file_path)

df

,Variable,Description
0,actualdpd_943P,Days Past Due (DPD) of previous contract (actu...
1,actualdpdtolerance_344P,DPD of client with tolerance.
2,addres_district_368M,District of the person's address.
3,addres_role_871L,Role of person's address.
4,addres_zip_823M,Zip code of the address.
...,...,...
460,totinstallast1m_4525188A,Total amount of monthly instalments paid in th...
461,twobodfilling_608L,Type of application process.
462,type_25L,Contact type of a person.
463,typesuite_864L,Persons accompanying the client during the loa...


In [ ]:
file_name = "csv_files/train/train_applprev_1_0.csv"

file_path = os.path.join(dataPath, file_name)

# Прочитайте CSV файл в DataFrame с помощью pandas
data1 = pd.read_csv(file_path, dtype={27: str})

data1

In [5]:
data1.describe()


,case_id,actualdpd_943P,annuity_853A,byoccupationinc_3656910L,childnum_21L,credacc_actualbalance_314A,credacc_credlmt_575A,credacc_maxhisbal_375A,credacc_minhisbal_90A,credacc_transactions_402L,credamount_590A,currdebt_94A,downpmt_134A,mainoccupationinc_437A,maxdpdtolerance_577P,num_group1,outstandingdebt_522A,pmtnum_8L,revolvingaccount_394A,tenor_203L
count,3.887684e+06,3.885450e+06,3.731833e+06,991660.000000,1.933791e+06,1.681780e+05,3.768614e+06,1.681780e+05,168178.000000,168178.000000,3.764355e+06,2.617307e+06,3.764355e+06,3.851072e+06,2.070306e+06,3.887684e+06,2.609762e+06,3.574851e+06,1.566510e+05,3.574851e+06
mean,1.397916e+06,1.056300e-02,3.413166e+03,19796.484030,8.434702e-01,2.026958e+04,3.254677e+03,-3.288887e+03,-6554.784203,0.522125,3.865786e+04,5.229101e+03,4.578899e+02,4.002969e+04,1.326582e+01,3.910262e+00,6.994588e+03,1.578210e+01,7.403548e+08,1.578210e+01
std,7.601594e+05,3.754277e+00,2.828269e+03,30687.652507,1.209425e+00,2.600278e+04,1.406135e+04,2.808611e+04,16888.862439,2.951672,3.754434e+04,1.927842e+04,2.697226e+03,3.139637e+04,1.348893e+02,4.101353e+00,2.916248e+04,1.046206e+01,5.498686e+07,1.046206e+01
min,2.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,-1.140860e+05,0.000000e+00,-1.961082e+05,-206808.170000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,5.403424e+08,3.000000e+00
25%,1.251506e+06,0.000000e+00,1.710600e+03,1.000000,0.000000e+00,3.030500e+00,0.000000e+00,0.000000e+00,-1042.079025,0.000000,1.399800e+04,0.000000e+00,0.000000e+00,1.800000e+04,0.000000e+00,1.000000e+00,0.000000e+00,6.000000e+00,7.406884e+08,6.000000e+00
50%,1.451959e+06,0.000000e+00,2.761200e+03,5000.000000,0.000000e+00,1.258500e+04,0.000000e+00,0.000000e+00,0.000000,0.000000,2.700000e+04,0.000000e+00,0.000000e+00,3.400000e+04,0.000000e+00,3.000000e+00,0.000000e+00,1.200000e+01,7.602487e+08,1.200000e+01
75%,1.641585e+06,0.000000e+00,4.396400e+03,30000.000000,1.000000e+00,3.044600e+04,0.000000e+00,0.000000e+00,0.000000,0.000000,5.000000e+04,0.000000e+00,0.000000e+00,5.200000e+04,1.000000e+00,6.000000e+00,0.000000e+00,2.400000e+01,7.607250e+08,2.400000e+01
max,2.651092e+06,3.676000e+03,1.051302e+05,200000.000000,2.000000e+01,2.540730e+06,4.000000e+05,7.988198e+06,199567.000000,155.000000,7.153920e+05,5.074297e+05,3.204000e+05,1.960000e+05,4.058000e+03,1.900000e+01,1.210629e+06,6.200000e+01,7.808654e+08,6.200000e+01


In [16]:
# dataPath = "/kaggle/input/home-credit-credit-risk-model-stability/"

def convert_strings(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:
        if df[col].dtype.name in ['object', 'string']:
            df[col] = df[col].astype("string").astype('category')
            current_categories = df[col].cat.categories
            new_categories = current_categories.to_list() + ["Unknown"]
            new_dtype = pd.CategoricalDtype(categories=new_categories, ordered=True)
            df[col] = df[col].astype(new_dtype)
    return df

train_basetable = pd.read_csv(dataPath + "csv_files/train/train_base.csv")
train_static = pd.concat(
    [
        pd.read_csv(dataPath + "csv_files/train/train_static_0_0.csv").pipe(convert_strings),
        pd.read_csv(dataPath + "csv_files/train/train_static_0_1.csv").pipe(convert_strings),
    ],
    axis=0,
    ignore_index=True
)
train_static_cb = pd.read_csv(dataPath + "csv_files/train/train_static_cb_0.csv").pipe(convert_strings)
train_person_1 = pd.read_csv(dataPath + "csv_files/train/train_person_1.csv").pipe(convert_strings)
train_credit_bureau_b_2 = pd.read_csv(dataPath + "csv_files/train/train_credit_bureau_b_2.csv").pipe(convert_strings)
test_basetable = pd.read_csv(dataPath + "csv_files/test/test_base.csv")
test_static = pd.concat(
    [
        pd.read_csv(dataPath + "csv_files/test/test_static_0_0.csv").pipe(convert_strings),
        pd.read_csv(dataPath + "csv_files/test/test_static_0_1.csv").pipe(convert_strings),
        pd.read_csv(dataPath + "csv_files/test/test_static_0_2.csv").pipe(convert_strings),
    ],
    axis=0,
    ignore_index=True
)
test_static_cb = pd.read_csv(dataPath + "csv_files/test/test_static_cb_0.csv").pipe(convert_strings)
test_person_1 = pd.read_csv(dataPath + "csv_files/test/test_person_1.csv").pipe(convert_strings)
test_credit_bureau_b_2 = pd.read_csv(dataPath + "csv_files/test/test_credit_bureau_b_2.csv").pipe(convert_strings)

C:\Users\Peter\AppData\Local\Temp\ipykernel_17768\3419565707.py:16: DtypeWarning: Columns (20,45,46,53,57,84,143,146,167) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(dataPath + "csv_files/train/train_static_0_0.csv").pipe(convert_strings),
C:\Users\Peter\AppData\Local\Temp\ipykernel_17768\3419565707.py:17: DtypeWarning: Columns (20,45,46,56,57,84,143,146,167) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(dataPath + "csv_files/train/train_static_0_1.csv").pipe(convert_strings),
C:\Users\Peter\AppData\Local\Temp\ipykernel_17768\3419565707.py:22: DtypeWarning: Columns (1,2,3,4,7,45,46,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  train_static_cb = pd.read_csv(dataPath + "csv_files/train/train_static_cb_0.csv").pipe(convert_strings)
C:\Users\Peter\AppData\Local\Temp\ipykernel_17768\3419565707.py:23: DtypeWarning: Columns (16) have mixed types. Specify dtype option on i

In [17]:
# We need to use aggregation functions in tables with depth > 1, so tables that contain num_group1 column or 
# also num_group2 column.
train_person_1_feats_1 = train_person_1.groupby("case_id").agg(
    {"mainoccupationinc_384A": "max",
     "incometype_1044T": lambda x: (x == "SELFEMPLOYED").max()}
).rename(columns={"mainoccupationinc_384A": "mainoccupationinc_384A_max",
                  "incometype_1044T": "mainoccupationinc_384A_any_selfemployed"})

# Here num_group1=0 has special meaning, it is the person who applied for the loan.
train_person_1_feats_2 = train_person_1[train_person_1["num_group1"] == 0][["case_id", "housetype_905L"]] \
    .rename(columns={"housetype_905L": "person_housetype"}).drop(columns=["num_group1"])

# Here we have num_goup1 and num_group2, so we need to aggregate again.
train_credit_bureau_b_2_feats = train_credit_bureau_b_2.groupby("case_id").agg(
    {"pmts_pmtsoverdue_635A": "max",
     "pmts_dpdvalue_108P": lambda x: (x > 31).max()}
).rename(columns={"pmts_pmtsoverdue_635A": "pmts_pmtsoverdue_635A_max",
                  "pmts_dpdvalue_108P": "pmts_dpdvalue_108P_over31"})

# We will process in this examples only A-type and M-type columns, so we need to select them.
selected_static_cols = [col for col in train_static.columns if col[-1] in ("A", "M")]
print(selected_static_cols)

selected_static_cb_cols = [col for col in train_static_cb.columns if col[-1] in ("A", "M")]
print(selected_static_cb_cols)

# Join all tables together.
data = train_basetable.merge(train_static[["case_id"] + selected_static_cols], how="left", on="case_id") \
    .merge(train_static_cb[["case_id"] + selected_static_cb_cols], how="left", on="case_id") \
    .merge(train_person_1_feats_1, how="left", on="case_id") \
    .merge(train_person_1_feats_2, how="left", on="case_id") \
    .merge(train_credit_bureau_b_2_feats, how="left", on="case_id")


KeyError: "['num_group1'] not found in axis"

In [18]:
train_person_1

,case_id,birth_259D,birthdate_87D,childnum_185L,contaddr_district_15M,contaddr_matchlist_1032L,contaddr_smempladdr_334L,contaddr_zipcode_807M,education_927M,empl_employedfrom_271D,...,registaddr_district_1083M,registaddr_zipcode_184M,relationshiptoclient_415T,relationshiptoclient_642T,remitter_829L,role_1084L,role_993L,safeguarantyflag_411L,sex_738L,type_25L
0,0,1986-07-01,NaN,NaN,P88_18_84,False,False,P167_100_165,P97_36_170,2017-09-15,...,P88_18_84,P167_100_165,NaN,NaN,NaN,CL,NaN,True,F,PRIMARY_MOBILE
1,0,NaN,NaN,NaN,a55475b1,NaN,NaN,a55475b1,a55475b1,NaN,...,a55475b1,a55475b1,SPOUSE,NaN,False,EM,NaN,NaN,NaN,PHONE
2,0,NaN,NaN,NaN,a55475b1,NaN,NaN,a55475b1,a55475b1,NaN,...,a55475b1,a55475b1,COLLEAGUE,SPOUSE,False,PE,NaN,NaN,NaN,PHONE
3,0,NaN,NaN,NaN,a55475b1,NaN,NaN,a55475b1,a55475b1,NaN,...,a55475b1,a55475b1,NaN,COLLEAGUE,NaN,PE,NaN,NaN,NaN,PHONE
4,1,1957-08-01,NaN,NaN,P103_93_94,False,False,P176_37_166,P97_36_170,2008-10-29,...,P103_93_94,P176_37_166,NaN,NaN,NaN,CL,NaN,True,M,PRIMARY_MOBILE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2973986,2703451,NaN,NaN,NaN,a55475b1,NaN,NaN,a55475b1,a55475b1,NaN,...,a55475b1,a55475b1,NaN,NaN,NaN,CL,NaN,NaN,NaN,HOME_PHONE
2973987,2703452,1977-08-01,NaN,NaN,P133_44_167,False,False,P59_150_74,a55475b1,NaN,...,P19_11_176,P11_15_81,NaN,NaN,NaN,CL,NaN,False,M,PRIMARY_MOBILE
2973988,2703453,1950-02-01,NaN,NaN,P123_6_84,False,False,P46_103_143,a55475b1,NaN,...,P123_6_84,P46_103_143,NaN,NaN,NaN,CL,NaN,False,F,PRIMARY_MOBILE
2973989,2703453,NaN,NaN,NaN,a55475b1,NaN,NaN,a55475b1,a55475b1,NaN,...,a55475b1,a55475b1,NaN,NaN,NaN,CL,NaN,NaN,NaN,HOME_PHONE
